In [132]:
#multi-agent testrun using pettingzoo
#still need to update the actionset
#still need to update reward function
#trial to see if it works

#update the actions so that it selects the frequency range of a particular radar first, and then picks a sub-band from inside it
#fix the frequency hopping of the radar so that it only transmits one frequency per radar per step.

#after finalizing actions and rewards, test on vanilla dqn and sstart implementing ddqn with per

#fixed the radar frquency issue, need to fix the states, obs, info etc tuple being returned by env to the dqn
#after that, train dqn
#finally move to ddqn with per

In [133]:
import numpy as np
import math
import gymnasium as gym
from gym import Env
from gym import spaces
import random
import numpy as np
from IPython.display import clear_output
import os
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

In [134]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)]]

interval = (freq_agile_radar_versions[0][1] - freq_agile_radar_versions[0][0])/40
print (interval)

hop_states = []
for i in range (0,40):
    
    hop_states.append(int(freq_agile_radar_versions[0][0]+ interval * i))

print (hop_states)
print (len(hop_states))


10000000.0
[9000000000, 9010000000, 9020000000, 9030000000, 9040000000, 9050000000, 9060000000, 9070000000, 9080000000, 9090000000, 9100000000, 9110000000, 9120000000, 9130000000, 9140000000, 9150000000, 9160000000, 9170000000, 9180000000, 9190000000, 9200000000, 9210000000, 9220000000, 9230000000, 9240000000, 9250000000, 9260000000, 9270000000, 9280000000, 9290000000, 9300000000, 9310000000, 9320000000, 9330000000, 9340000000, 9350000000, 9360000000, 9370000000, 9380000000, 9390000000]
40


In [135]:
#hop_states = np.linspace(9e9, 9.4e9, 20e6)
jamming_bandwidths = [10e6, 20e6, 30e6]

In [136]:
from pettingzoo import ParallelEnv
from gym import spaces
import numpy as np

class ParallelRadarJammingEnv(ParallelEnv):
    def __init__(self, hop_states, jamming_bandwidths, max_hop_length=8):
        super().__init__()

        self.hop_states = hop_states
        self.low = self.hop_states[0]
        self.interval = self.hop_states[1] - self.hop_states[0]
        self.n_frequencies = len(hop_states)
        self.max_hop_length = max_hop_length

        # agent params, action spaces and observation spaces
        self.agents = ["jammer_0", "jammer_1", "jammer_2", "jammer_3", "jammer_4", "jammer_5"]
        self.possible_agents = self.agents[:]
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}

        self.jamming_bandwidths = jamming_bandwidths
        self.n_bandwidths = len(self.jamming_bandwidths)
        self.action_space = {agent: spaces.Discrete(self.max_hop_length * self.n_bandwidths) for agent in self.agents}

        self.observation_spaces = {
            agent: spaces.Box(low=0, high=1, shape=(self.n_frequencies,), dtype=np.float32) for agent in self.agents
        }
        print(self.observation_spaces)
        # step and reward vars
        self.hopping_patterns = None
        self.current_frequencies = None
        self.current_steps = None
        self.cumulative_rewards = None
        self.net_agent_rewards = None
        self.terminations = None
        self.rewards = None
        self.info = None

    def index_to_action(self, index):
        frequency_index = index // self.n_bandwidths
        bandwidth_index = index % self.n_bandwidths
        frequency = int(self.low + frequency_index * self.interval)
        bandwidth = self.jamming_bandwidths[bandwidth_index]
        return frequency, bandwidth


    def generate_hopping_pattern(self):
        return np.random.choice(self.hop_states, self.max_hop_length, replace=False)

    def next_frequency(self, agent):
        return self.hopping_patterns[agent][self.current_steps[agent] % self.max_hop_length]

    def reset(self):

        #set radar freq
        self.radar_frequency = self.generate_hopping_pattern()[0]

        #init states for agents
        self.current_steps = {agent: 0 for agent in self.agents}
        self.cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.rewards = {agent: 0 for agent in self.agents}
        self.info = {agent: {} for agent in self.agents}

        #obs for agents
        observations = {agent: self.observe(agent) for agent in self.agents}
        return observations


    def observe(self, agent):

        #one-hot encoded obs of the radar freqs used
        observation = np.zeros(self.n_frequencies)
        freq_index = np.where(self.hop_states == self.radar_frequency)[0][0]
        observation[freq_index] = 1
        return observation


    def step(self, actions):
        total_jammed_frequencies = 0
        self.net_agent_rewards = 0

        for agent, action in actions.items():

            frequency, bandwidth = self.index_to_action(action)

            lower_bound = frequency - bandwidth
            upper_bound = frequency + bandwidth

            radar_frequency = self.radar_frequency

            if lower_bound <= radar_frequency <= upper_bound:
                reward = 100
                total_jammed_frequencies += 1
            else:
                reward = -50

            if 2 * (bandwidth / self.interval) > 5:
                reward -= 1
            else:
                reward += 0

            jam_threshold = len(self.hop_states) / len(self.agents)
            if total_jammed_frequencies > jam_threshold:
                reward += 5
            else:
                reward -= 5

            #update rewards
            self.current_steps[agent] += 1
            self.cumulative_rewards[agent] += reward
            self.net_agent_rewards += reward

            #num steps per episode
            if self.current_steps[agent] >= 100:
                self.terminations[agent] = True

            self.rewards[agent] = reward

        #update radar req for next step
        self.radar_frequency = self.generate_hopping_pattern()[0]

        #return the observations, rewards, done status, and info for each agent
        observations = {agent: self.observe(agent) for agent in self.agents}
        rewards = {agent: self.rewards[agent] for agent in self.agents}
        done = {agent: self.terminations[agent] for agent in self.agents}
        info = {agent: self.info[agent] for agent in self.agents}

        return observations, rewards, done, info

    def render(self):
        for agent in self.agents:
            print(f"Agent {agent}, Step: {self.current_steps[agent]}, Radar Frequency: {self.radar_frequency}")


In [137]:
env = ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

observations = env.reset()
done = {agent: False for agent in env.agents}
cumulative_reward = {agent: 0 for agent in env.agents}

while not all(done.values()):

    actions = {agent: env.action_space[agent].sample() for agent in env.agents}
    observations, rewards, done, infos = env.step(actions)

    for agent in env.agents:
        cumulative_reward[agent] += rewards[agent]

    env.render()
    print(f"Actions: {actions}, Rewards: {rewards}, Cumulative Rewards: {cumulative_reward}")
    print('\n')


{'jammer_0': Box(0.0, 1.0, (40,), float32), 'jammer_1': Box(0.0, 1.0, (40,), float32), 'jammer_2': Box(0.0, 1.0, (40,), float32), 'jammer_3': Box(0.0, 1.0, (40,), float32), 'jammer_4': Box(0.0, 1.0, (40,), float32), 'jammer_5': Box(0.0, 1.0, (40,), float32)}
Agent jammer_0, Step: 1, Radar Frequency: 9240000000
Agent jammer_1, Step: 1, Radar Frequency: 9240000000
Agent jammer_2, Step: 1, Radar Frequency: 9240000000
Agent jammer_3, Step: 1, Radar Frequency: 9240000000
Agent jammer_4, Step: 1, Radar Frequency: 9240000000
Agent jammer_5, Step: 1, Radar Frequency: 9240000000
Actions: {'jammer_0': 15, 'jammer_1': 10, 'jammer_2': 17, 'jammer_3': 11, 'jammer_4': 11, 'jammer_5': 12}, Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -56, 'jammer_3': -56, 'jammer_4': -56, 'jammer_5': -55}, Cumulative Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -56, 'jammer_3': -56, 'jammer_4': -56, 'jammer_5': -55}


Agent jammer_0, Step: 2, Radar Frequency: 9120000000
Agent jammer_1, Step: 2, 

In [138]:
#setting up replay memory

Transition = namedtuple('Transition',
('state', 'action', 'next_state', 'reward')
)

In [139]:
class ReplayMemory (object):

    def __init__(self, capacity):

        self.memory = deque([], maxlen = capacity)

    def push (self, *args):

        # save a transition
        self.memory.append(Transition(*args))

    def sample (self, batch_size):

        return random.sample (self.memory, batch_size)

    def __len__ (self):

        return len(self.memory)

In [140]:
class DQN (nn.Module):

    def __init__ (self, n_observations, n_actions):

        super (DQN, self).__init__()

        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear (128, n_actions)

    def forward (self, x):

        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))

        return self.layer3(x)

In [143]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

n_actions = env.action_space['jammer_0']
observations, rewards, done, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0

{'jammer_0': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 'jammer_1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 'jammer_2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 'jammer_3': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 'jammer_4': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 'jammer_5': array([0., 0

ValueError: too many values to unpack (expected 4)

In [ ]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [ ]:
episode_rewards = []

In [ ]:
def plot_rewards(show_result=False):
    plt.figure(1)
    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Rewards')
    plt.plot(rewards_t.numpy())
    # Take 100 episode averages and plot them too
    if len(rewards_t) >= 100:
        means = rewards_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.1)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())


In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 2000
    

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    #t is the step number
    #count is the iterator over num steps
    cumulative_reward = 0
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        cumulative_reward += reward
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_rewards.append(cumulative_reward)
            plot_rewards()
            break

print('Complete')
plot_rewards(show_result=True)
plt.ioff()
plt.show()